In [2]:
# import it 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# load the dataset
file_path = "liv_midfield_scouting_fbref.xlsx"
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names
print("Available sheets:", sheet_names)

Available sheets: ['alexis_mcallister', 'curits_jones', 'ryan_gravenberch', 'dominik_szoboszlai']
